<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/Factor%20Model/5_Factor_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 Factor Model 

In [93]:
import pandas as pd
import numpy as np

In [94]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Factor%20Model/MSCI_Excess_Earn_Five_Factor.csv')
df.head()

,Month,Stock_Name,Intercept,Mkt-RF,Mkt-RF_coef,SMB,SMB_coef,HML,HML_coef,RMW,RMW_coef,CMA,CMA_coef,Excess_Return,RF,Expected_Return,Company_name,Stock_exchange,Industry,SIC_Code,SIC_2,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,Longterm_BV,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,Longterm_MV,Longterm_BMR,Industry_Ave_BMR,BookValue_Quarterly,MarketValue_Quarterly,BMR_Quarterly,Industry_adjusted_BMR_Quarterly,Size,Momentum,ESG_Rating,ESG_Score,Size_G,BMR_G,Mom_G,ESG_G,Monthly_Return,Porfolio_Return,ESG_Portfolio_Return,ESG_Abnormal_Return,Excess_Earn
0,1,A,0.079478,-0.11,1.329246,-4.4,-0.437637,-6.27,-0.137567,-1.36,-0.314086,-2.34,-0.295079,3.839049,0.13,3.969049,Agilent Technologies Inc,NYSE,Healthcare,3826,38,4243.000,4831.000,4567.000,4748.000,4873.000,4652.400,13888.064911,14658.833458,21632.517268,21488.239823,2.646175e+04,19625.880519,0.237054,0.307958,4848.000,2.646175e+04,0.183208,-0.124750,26461.710,11.63787,AAA,8.8,5,3,2,Leader_Group,-3.223538,-2.903777,-5.848175,-2.944398,-7.192587
1,1,AAL,-2.075022,-0.11,1.462907,-4.4,0.293291,-6.27,0.977270,-1.36,2.353334,-2.34,-0.826417,-10.920623,0.13,-10.790623,American Airlines Group Inc,NASDAQ,Industrials,4512,45,5635.000,3785.000,-780.000,-169.000,-118.000,1670.600,26694.286577,24191.263534,24896.306768,14789.270777,1.256350e+04,20626.926053,0.080991,0.618105,-118.000,1.256350e+04,-0.009392,-0.627498,12280.780,-28.43626,CCC,0.7,4,1,1,Laggard_Group,-6.415624,-3.839508,-8.223340,-4.383832,4.374999
2,1,AAON,0.686283,-0.11,0.675028,-4.4,0.883944,-6.27,0.289513,-1.36,1.543038,-2.34,-0.626584,-5.724897,0.13,-5.594897,Aaon Inc,NASDAQ,Industrials,3585,35,178.918,205.898,237.226,249.443,290.140,232.325,1256.035791,1743.560682,1925.589326,1826.434467,2.573199e+03,1864.963938,0.124573,0.360987,290.140,2.573199e+03,0.112755,-0.248233,2574.162,30.09222,A,5.8,2,1,4,Average_Group,6.132364,-0.441837,1.451676,1.893513,11.727261
3,1,AAP,-0.686524,-0.11,1.003027,-4.4,0.467205,-6.27,-0.379270,-1.36,1.979816,-2.34,2.009285,-7.868808,0.13,-7.738808,Advance Auto Parts Inc,NYSE,Consumer Cyclicals,5531,55,2460.648,2916.192,3415.196,3550.813,3549.081,3178.386,11022.840214,12456.301060,7366.895907,11477.844994,1.109258e+04,10683.293408,0.297510,0.375682,3549.081,1.109258e+04,0.319951,-0.055731,11092.590,-11.60880,BBB,4.6,4,3,1,Average_Group,-17.738520,-4.628674,-9.200801,-4.572127,-9.999712
4,1,AAPL,0.041151,-0.11,0.924850,-4.4,0.129033,-6.27,0.170799,-1.36,1.499483,-2.34,-1.812227,0.502075,0.13,0.632075,Apple Inc,NASDAQ,Technology,3663,36,128249.000,134047.000,107147.000,90488.000,65339.000,105054.000,583612.701620,608960.249120,868879.619760,748539.080520,1.304765e+06,822951.283654,0.127655,0.395727,89531.000,1.304765e+06,0.068618,-0.327109,1304765.000,49.65282,A,6.4,5,1,5,Average_Group,5.400991,1.921124,3.635589,1.714465,4.768916


In [95]:
df =df[['Stock_Name',	'Month',	'Size',	'Momentum', 'Size_G',	'BMR_G',	'Mom_G','Monthly_Return','Expected_Return']]
df.head()

,Stock_Name,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Expected_Return
0,A,1,26461.710,11.63787,5,3,2,-3.223538,3.969049
1,AAL,1,12280.780,-28.43626,4,1,1,-6.415624,-10.790623
2,AAON,1,2574.162,30.09222,2,1,4,6.132364,-5.594897
3,AAP,1,11092.590,-11.60880,4,3,1,-17.738520,-7.738808
4,AAPL,1,1304765.000,49.65282,5,1,5,5.400991,0.632075


In [96]:
df['Abnormal_Return']=df['Monthly_Return']-df['Expected_Return']
df.head()

,Stock_Name,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Expected_Return,Abnormal_Return
0,A,1,26461.710,11.63787,5,3,2,-3.223538,3.969049,-7.192587
1,AAL,1,12280.780,-28.43626,4,1,1,-6.415624,-10.790623,4.374999
2,AAON,1,2574.162,30.09222,2,1,4,6.132364,-5.594897,11.727261
3,AAP,1,11092.590,-11.60880,4,3,1,-17.738520,-7.738808,-9.999712
4,AAPL,1,1304765.000,49.65282,5,1,5,5.400991,0.632075,4.768916


In [97]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Factor%20Model/ESG_score_2020Jan.csv')
ESG.head()

,Symbol,Name,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance
0,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9
1,AAL,"American Airlines Group, Inc.",CCC,0.7,3.3,2.5,5.7
2,AAN,Aaron's Company Inc,@NA,NaN,NaN,NaN,NaN
3,AAOI,"Applied Optoelectronics, Inc.",@NA,NaN,NaN,NaN,NaN
4,AAON,"AAON, Inc.",A,5.8,5.5,8.4,5.7


In [98]:
ESG.isna().sum()

Symbol                                                   0
Name                                                     0
MSCI ESG Rtg Overall Company Rating                      0
MSCI ESG Rtg Overall Industry-Adjusted Company Score    32
MSCI ESG Rtg Score Environmental                        32
MSCI ESG Rtg Score Social                               32
MSCI ESG Rtg Score Governance                           32
dtype: int64

In [99]:
ESG=ESG.dropna()

In [100]:
ESG['ESG_G']=pd.qcut(ESG['MSCI ESG Rtg Overall Industry-Adjusted Company Score'],5,labels=np.arange(1,6,1))
ESG['E_G']=pd.qcut(ESG['MSCI ESG Rtg Score Environmental'],5,labels=np.arange(1,6,1))
ESG['S_G']=pd.qcut(ESG['MSCI ESG Rtg Score Social'],5,labels=np.arange(1,6,1))
ESG['G_G']=pd.qcut(ESG['MSCI ESG Rtg Score Governance'],5,labels=np.arange(1,6,1))

In [101]:
ESG.head()

,Symbol,Name,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance,ESG_G,E_G,S_G,G_G
0,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3
1,AAL,"American Airlines Group, Inc.",CCC,0.7,3.3,2.5,5.7,1,2,1,3
4,AAON,"AAON, Inc.",A,5.8,5.5,8.4,5.7,4,4,5,3
5,AAP,"Advance Auto Parts, Inc.",BBB,4.6,2.0,3.8,6.6,3,1,2,4
6,AAPL,Apple Inc.,A,6.4,3.6,5.8,4.8,4,2,5,2


In [102]:
ESG['ESG_Group']=ESG['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['E_Group']=ESG['E_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['S_Group']=ESG['S_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['G_Group']=ESG['G_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')

In [103]:
df=df.merge(ESG, left_on='Stock_Name', right_on='Symbol')
df.head()

,Stock_Name,Month,Size,Momentum,Size_G,BMR_G,Mom_G,Monthly_Return,Expected_Return,Abnormal_Return,Symbol,Name,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance,ESG_G,E_G,S_G,G_G,ESG_Group,E_Group,S_Group,G_Group
0,A,1,26461.71,11.63787,5,3,2,-3.223538,3.969049,-7.192587,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
1,A,10,31120.78,41.21783,5,3,5,0.859511,-3.288307,4.147818,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
2,A,11,31475.33,34.87440,5,3,5,14.506820,12.107495,2.399325,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
3,A,12,35771.40,32.63608,5,3,5,2.713239,5.014762,-2.301523,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average
4,A,2,25608.71,26.46012,5,3,2,-6.649709,-8.840391,2.190682,A,"Agilent Technologies, Inc.",AAA,8.8,10.0,6.6,5.9,5,5,5,3,Leader,Leader,Leader,Average


#Virtualization

In [104]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.103657,8.451522,816,28.565714,0.295862
1,1,Laggard,-0.197932,10.194556,259,16.093477,0.633459
2,1,Leader,0.960673,6.776895,252,15.874508,0.426904
3,2,Average,-1.557311,9.493744,816,28.565714,0.332348
4,2,Laggard,0.082846,12.042227,259,16.093477,0.748268


In [105]:
import plotly.express as px
fig1 = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig1.show()

In [106]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [107]:
import plotly.express as px
fig2 = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig2.show()

In [108]:
E_summary = df.groupby(by=['Month','E_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
E_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
E_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
E_summary.head()

,Month,E_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.947130,8.843689,839,NaN,NaN
1,1,Laggard,-1.258656,8.078621,259,16.093477,0.633459
2,1,Leader,-0.843848,7.580517,229,15.874508,0.426904
3,2,Average,-1.307613,10.131203,839,NaN,NaN
4,2,Laggard,-1.091474,9.588389,259,16.093477,0.748268


In [109]:
E_summary = E_summary[E_summary['E_Group'].isin(['Leader', 'Laggard'])]

In [110]:
import plotly.express as px
figE = px.line(E_summary, x="Month", y="mean", color="E_Group", error_y="mean_std")
figE.show()

In [111]:
S_summary = df.groupby(by=['Month','S_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
S_summary['sqrt_count']=S_summary['count'].apply(lambda x: np.sqrt(x))
S_summary['mean_std']=S_summary['std']/S_summary['sqrt_count']
S_summary.head()

,Month,S_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.180846,8.615885,820,28.635642,0.300880
1,1,Laggard,0.778719,8.627989,259,16.093477,0.536117
2,1,Leader,0.895219,8.153573,248,15.748016,0.517752
3,2,Average,-1.391021,9.498550,820,28.635642,0.331704
4,2,Laggard,-0.716296,10.218893,259,16.093477,0.634971


In [112]:
S_summary = S_summary[S_summary['S_Group'].isin(['Leader', 'Laggard'])]

In [113]:
import plotly.express as px
figS = px.line(S_summary, x="Month", y="mean", color="S_Group", error_y="mean_std")
figS.show()

In [114]:
G_summary = df.groupby(by=['Month','G_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
G_summary['sqrt_count']=G_summary['count'].apply(lambda x: np.sqrt(x))
G_summary['mean_std']=G_summary['std']/G_summary['sqrt_count']
G_summary.head()

,Month,G_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.206398,8.788262,800,28.284271,0.310712
1,1,Laggard,-0.054745,9.010111,292,17.088007,0.527277
2,1,Leader,0.537344,6.970717,235,15.329710,0.454719
3,2,Average,-0.929638,9.662780,800,28.284271,0.341631
4,2,Laggard,-1.016135,12.055497,292,17.088007,0.705495


In [115]:
G_summary = G_summary[G_summary['G_Group'].isin(['Leader', 'Laggard'])]

In [116]:
import plotly.express as px
figG = px.line(G_summary, x="Month", y="mean", color="G_Group", error_y="mean_std")
figG.show()

# T-Test

In [117]:
pip install pingouin

In [118]:
import pingouin as pg

In [119]:
Leader =df[df['ESG_Group']=='Leader']['Abnormal_Return']
Laggard=df[df['ESG_Group']=='Laggard']['Abnormal_Return']

In [120]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.462802,5977.324788,two-sided,0.143574,"[-0.16, 1.13]",0.037269,0.084,0.308455


In [121]:
E_Leader =df[df['E_Group']=='Leader']['Abnormal_Return']
E_Laggard=df[df['E_Group']=='Laggard']['Abnormal_Return']

In [122]:
t_test = pg.ttest(E_Leader, E_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-2.900717,5757.286574,two-sided,0.003737,"[-1.58, -0.3]",0.074799,1.955,0.814911


In [123]:
S_Leader =df[df['S_Group']=='Leader']['Abnormal_Return']
S_Laggard=df[df['S_Group']=='Laggard']['Abnormal_Return']

In [124]:
t_test = pg.ttest(S_Leader, S_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.033419,6071.224429,two-sided,0.301449,"[-0.31, 0.99]",0.026503,0.049,0.178412


In [125]:
G_Leader =df[df['G_Group']=='Leader']['Abnormal_Return']
G_Laggard=df[df['G_Group']=='Laggard']['Abnormal_Return']

In [126]:
t_test = pg.ttest(G_Leader, G_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.115599,6321.307167,two-sided,0.264636,"[-0.27, 0.98]",0.027607,0.053,0.193602


In [127]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Factor%20Model/MSCI_Excess_Earn_Five_Factor.csv')
df.head()

,Month,Stock_Name,Intercept,Mkt-RF,Mkt-RF_coef,SMB,SMB_coef,HML,HML_coef,RMW,RMW_coef,CMA,CMA_coef,Excess_Return,RF,Expected_Return,Company_name,Stock_exchange,Industry,SIC_Code,SIC_2,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,Longterm_BV,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,Longterm_MV,Longterm_BMR,Industry_Ave_BMR,BookValue_Quarterly,MarketValue_Quarterly,BMR_Quarterly,Industry_adjusted_BMR_Quarterly,Size,Momentum,ESG_Rating,ESG_Score,Size_G,BMR_G,Mom_G,ESG_G,Monthly_Return,Porfolio_Return,ESG_Portfolio_Return,ESG_Abnormal_Return,Excess_Earn
0,1,A,0.079478,-0.11,1.329246,-4.4,-0.437637,-6.27,-0.137567,-1.36,-0.314086,-2.34,-0.295079,3.839049,0.13,3.969049,Agilent Technologies Inc,NYSE,Healthcare,3826,38,4243.000,4831.000,4567.000,4748.000,4873.000,4652.400,13888.064911,14658.833458,21632.517268,21488.239823,2.646175e+04,19625.880519,0.237054,0.307958,4848.000,2.646175e+04,0.183208,-0.124750,26461.710,11.63787,AAA,8.8,5,3,2,Leader_Group,-3.223538,-2.903777,-5.848175,-2.944398,-7.192587
1,1,AAL,-2.075022,-0.11,1.462907,-4.4,0.293291,-6.27,0.977270,-1.36,2.353334,-2.34,-0.826417,-10.920623,0.13,-10.790623,American Airlines Group Inc,NASDAQ,Industrials,4512,45,5635.000,3785.000,-780.000,-169.000,-118.000,1670.600,26694.286577,24191.263534,24896.306768,14789.270777,1.256350e+04,20626.926053,0.080991,0.618105,-118.000,1.256350e+04,-0.009392,-0.627498,12280.780,-28.43626,CCC,0.7,4,1,1,Laggard_Group,-6.415624,-3.839508,-8.223340,-4.383832,4.374999
2,1,AAON,0.686283,-0.11,0.675028,-4.4,0.883944,-6.27,0.289513,-1.36,1.543038,-2.34,-0.626584,-5.724897,0.13,-5.594897,Aaon Inc,NASDAQ,Industrials,3585,35,178.918,205.898,237.226,249.443,290.140,232.325,1256.035791,1743.560682,1925.589326,1826.434467,2.573199e+03,1864.963938,0.124573,0.360987,290.140,2.573199e+03,0.112755,-0.248233,2574.162,30.09222,A,5.8,2,1,4,Average_Group,6.132364,-0.441837,1.451676,1.893513,11.727261
3,1,AAP,-0.686524,-0.11,1.003027,-4.4,0.467205,-6.27,-0.379270,-1.36,1.979816,-2.34,2.009285,-7.868808,0.13,-7.738808,Advance Auto Parts Inc,NYSE,Consumer Cyclicals,5531,55,2460.648,2916.192,3415.196,3550.813,3549.081,3178.386,11022.840214,12456.301060,7366.895907,11477.844994,1.109258e+04,10683.293408,0.297510,0.375682,3549.081,1.109258e+04,0.319951,-0.055731,11092.590,-11.60880,BBB,4.6,4,3,1,Average_Group,-17.738520,-4.628674,-9.200801,-4.572127,-9.999712
4,1,AAPL,0.041151,-0.11,0.924850,-4.4,0.129033,-6.27,0.170799,-1.36,1.499483,-2.34,-1.812227,0.502075,0.13,0.632075,Apple Inc,NASDAQ,Technology,3663,36,128249.000,134047.000,107147.000,90488.000,65339.000,105054.000,583612.701620,608960.249120,868879.619760,748539.080520,1.304765e+06,822951.283654,0.127655,0.395727,89531.000,1.304765e+06,0.068618,-0.327109,1304765.000,49.65282,A,6.4,5,1,5,Average_Group,5.400991,1.921124,3.635589,1.714465,4.768916


In [128]:
Leader =df[df['ESG_G']=='Leader_Group'] ['Excess_Earn']
Laggard=df[df['ESG_G']=='Laggard_Group']['Excess_Earn']

In [129]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.468525,6035.469315,two-sided,0.142014,"[-0.16, 1.13]",0.037141,0.084,0.310169


In [130]:
ESG_summary = df.groupby(by=['Month','ESG_G'],as_index=False)['Excess_Earn'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_G,mean,std,count,sqrt_count,mean_std
0,1,Average_Group,0.114220,8.451737,817,28.583212,0.295689
1,1,Laggard_Group,-0.267138,10.145248,263,16.217275,0.625583
2,1,Leader_Group,0.998372,6.752126,255,15.968719,0.422835
3,2,Average_Group,-1.557699,9.487931,817,28.583212,0.331941
4,2,Laggard_Group,0.484263,13.741053,263,16.217275,0.847310


In [131]:
ESG_summary = ESG_summary[ESG_summary['ESG_G'].isin(['Leader_Group', 'Laggard_Group'])]

In [132]:
import plotly.express as px
fig1 = px.line(ESG_summary, x="Month", y="mean", color="ESG_G", error_y="mean_std")
fig1.show()